<a href="https://colab.research.google.com/github/emkafie/Machine-Learning/blob/main/TP_JS09_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tugas 2**

## **Langkah 1: LOAD & PREPROCESSING DATA**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [3]:
# ==========================================
# 1. LOAD & PREPROCESSING DATA
# ==========================================
# Memuat data
df = pd.read_csv('spam.csv', encoding='latin-1')

# Drop kolom yang tidak perlu
df = df.drop(df.iloc[:, 2:], axis=1)

# Rename kolom agar lebih jelas
df.rename(columns={'v1': 'Labels', 'v2': 'SMS'}, inplace=True)

# Encoding Label (spam=1, ham=0)
new_labels = {'spam': 1, 'ham': 0}
df['Labels'] = df['Labels'].map(new_labels)

# Memisahkan Fitur dan Label
X = df['SMS'].values
y = df['Labels'].values

# Split Data (80% Train, 20% Test)
# Menggunakan random_state yang sama agar perbandingan adil
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)

## **COUNTVECTORIZER DENGAN STOP WORDS**

In [4]:
print("--- EVALUASI MODEL 1: COUNTVECTORIZER ---")

# Inisiasi CountVectorizer dengan stop_words enabled
cv = CountVectorizer(stop_words='english')

# Fitting dan Transform
X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

# Training Model Multinomial Naive Bayes
mnb_cv = MultinomialNB()
mnb_cv.fit(X_train_cv, y_train)

# Prediksi & Evaluasi
y_pred_cv = mnb_cv.predict(X_test_cv)
acc_cv = accuracy_score(y_test, y_pred_cv)

print(f"Akurasi (CountVectorizer): {acc_cv:.4f}")
print("Laporan Klasifikasi:\n", classification_report(y_test, y_pred_cv))

--- EVALUASI MODEL 1: COUNTVECTORIZER ---
Akurasi (CountVectorizer): 0.9830
Laporan Klasifikasi:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99       954
           1       0.98      0.90      0.94       161

    accuracy                           0.98      1115
   macro avg       0.98      0.95      0.96      1115
weighted avg       0.98      0.98      0.98      1115



## **TF-IDF DENGAN STOP WORDS**

In [5]:
print("\n--- EVALUASI MODEL 2: TF-IDF ---")

# Inisiasi TfidfVectorizer dengan stop_words enabled
tfidf = TfidfVectorizer(stop_words='english')

# Fitting dan Transform
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Training Model Multinomial Naive Bayes
mnb_tfidf = MultinomialNB()
mnb_tfidf.fit(X_train_tfidf, y_train)

# Prediksi & Evaluasi
y_pred_tfidf = mnb_tfidf.predict(X_test_tfidf)
acc_tfidf = accuracy_score(y_test, y_pred_tfidf)

print(f"Akurasi (TF-IDF): {acc_tfidf:.4f}")
print("Laporan Klasifikasi:\n", classification_report(y_test, y_pred_tfidf))


--- EVALUASI MODEL 2: TF-IDF ---
Akurasi (TF-IDF): 0.9605
Laporan Klasifikasi:
               precision    recall  f1-score   support

           0       0.96      1.00      0.98       954
           1       1.00      0.73      0.84       161

    accuracy                           0.96      1115
   macro avg       0.98      0.86      0.91      1115
weighted avg       0.96      0.96      0.96      1115



## **PERBANDINGAN**

In [6]:
print("\n--- PERBANDINGAN HASIL ---")
print(f"Akurasi CountVectorizer : {acc_cv:.4f}")
print(f"Akurasi TF-IDF          : {acc_tfidf:.4f}")


--- PERBANDINGAN HASIL ---
Akurasi CountVectorizer : 0.9830
Akurasi TF-IDF          : 0.9605


### **Kesimpulan Analisis**

Stop Words: Penggunaan stop_words='english' pada kedua metode berfungsi untuk menghilangkan kata-kata umum (seperti "the", "is", "at") yang tidak memiliki makna spesifik dalam menentukan apakah sebuah pesan adalah spam atau bukan. Hal ini membantu mengurangi dimensi fitur dan kebisingan (noise) data.

Perbandingan Akurasi:

* Model dengan CountVectorizer cenderung menghasilkan akurasi yang sedikit lebih tinggi atau setara dibandingkan model dengan TF-IDF pada kasus deteksi spam SMS ini.

* Model dengan TF-IDF seringkali memiliki performa sedikit di bawah CountVectorizer dalam mendeteksi kelas minoritas (Spam) pada dataset teks pendek seperti SMS. Hal ini karena TF-IDF memberikan bobot lebih rendah pada kata yang sering muncul. Padahal, dalam kasus Spam, kata-kata kunci tertentu (misal: "Free", "Win") mungkin sering muncul di banyak data latih spam dan frekuensi kemunculan tersebut justru menjadi indikator kuat.

Kesimpulan Fitur Terbaik: Untuk kasus dataset spam.csv ini, fitur CountVectorizer (Bag of Words) terbukti lebih optimal dibandingkan TF-IDF. Alasannya adalah teks SMS cenderung pendek dan penggunaan kata-kata spam sangat spesifik. Frekuensi kemunculan kata secara mentah (raw count) yang disediakan oleh CountVectorizer sudah cukup kuat untuk membedakan antara spam dan ham tanpa perlu pembobotan invers dokumen yang dilakukan oleh TF-IDF.